# 非线性结构：图

## 树与图的区别:
- 图是由顶点集合以及顶点间的关系集合组成的一种数据结构:$Graph＝( V, E ), V = { x }, E = {(x, y) | x, y 属于 V }   $
- 树中除根节点外每个节点都只有一个先驱（上一层元素）
- 图中任意两个节点都可能建立直接关系
- 树是一种层次结构，图是网格结构
- 树是一种图的特例

## 图的基本概念：
- 路径：由边连接的顶点路径
    - 简单路径：若路径上各顶点 v1, v2, ..., vm 均不互相重复, 则称这样的路径为简单路径
    - 连通图与连通分量：在无向图中, 若从顶点v1到顶点v2有路径, 则称顶点v1与v2是连通的。如果图中任意一对顶点都是连通的, 则称此图是连通图。非连通图的极大连通子图叫做连通分量
    - 强连通图与强连通分量：在有向图中, 若对于每一对顶点vi和vj, 都存在一条从vi到vj和从vj到vi的路径, 则称此图是强连通图。非强连通图的极大强连通子图叫做强连通分量
- 环路：起点和终点相同的路径
- 有向图(digraph)：顶点对 <x, y> 是有序的
- 无向图(undigraph)：顶点对(x, y)是无序的
- 带权网络(weighted network)：每条边有指定的权重，记作G(V, E, wt())

## 图的基本实现：

In [2]:
import sys
from enum import Enum

# 定义顶点状态枚举
class VStatus(Enum):
    UNDISCOVERED = 0
    DISCOVERED = 1
    VISITED = 2

# 定义边类型枚举
class EType(Enum):
    UNDETERMINED = 0
    TREE = 1
    CROSS = 2
    FORWARD = 3
    BACKWARD = 4

class Vertex:
    """顶点类"""
    def __init__(self, data=None):
        self.data = data if data is not None else 0
        self.in_degree = 0
        self.out_degree = 0
        self.status = VStatus.UNDISCOVERED
        self.d_time = -1
        self.f_time = -1
        self.parent = -1
        self.priority = sys.maxsize

class Edge:
    """边类"""
    def __init__(self, data=None, weight=0):
        self.data = data
        self.weight = weight
        self.type = EType.UNDETERMINED

class GraphMatrix:
    """基于邻接矩阵的图类"""
    def __init__(self):
        self.vertices = []  # 顶点列表
        self.edges = []     # 邻接矩阵，存储边的引用
        self.n = 0          # 顶点总数
        self.e = 0          # 边总数
    
    def __del__(self):
        """析构函数，释放所有边对象"""
        for i in range(self.n):
            for j in range(self.n):
                if self.edges[i][j] is not None:
                    del self.edges[i][j]
                    self.edges[i][j] = None
    
    def exists(self, i, j):
        """判断边(i, j)是否存在"""
        if not (0 <= i < self.n and 0 <= j < self.n):
            return False
        if i >= len(self.edges) or j >= len(self.edges[i]):
            return False
        return self.edges[i][j] is not None
    
    def edge_type(self, i, j):
        """获取边(i, j)的类型"""
        if self.exists(i, j):
            return self.edges[i][j].type
        return None
    
    def set_edge_type(self, i, j, edge_type):
        """设置边(i, j)的类型"""
        if self.exists(i, j):
            self.edges[i][j].type = edge_type
    
    def edge_data(self, i, j):
        """获取边(i, j)的数据"""
        if self.exists(i, j):
            return self.edges[i][j].data
        return None
    
    def set_edge_data(self, i, j, data):
        """设置边(i, j)的数据"""
        if self.exists(i, j):
            self.edges[i][j].data = data
    
    def edge_weight(self, i, j):
        """获取边(i, j)的权重"""
        if self.exists(i, j):
            return self.edges[i][j].weight
        return None
    
    def set_edge_weight(self, i, j, weight):
        """设置边(i, j)的权重"""
        if self.exists(i, j):
            self.edges[i][j].weight = weight
    
    def vertex_data(self, i):
        """获取顶点i的数据"""
        if 0 <= i < self.n:
            return self.vertices[i].data
        return None
    
    def set_vertex_data(self, i, data):
        """设置顶点i的数据"""
        if 0 <= i < self.n:
            self.vertices[i].data = data
    
    def in_degree(self, i):
        """获取顶点i的入度"""
        if 0 <= i < self.n:
            return self.vertices[i].in_degree
        return -1
    
    def out_degree(self, i):
        """获取顶点i的出度"""
        if 0 <= i < self.n:
            return self.vertices[i].out_degree
        return -1
    
    def first_neighbor(self, i):
        """获取顶点i的第一个邻接顶点"""
        return self.next_neighbor(i, self.n)
    
    def next_neighbor(self, i, j):
        """获取顶点i在j之后的下一个邻接顶点"""
        if not (0 <= i < self.n and 0 <= j <= self.n):
            return -1
        j -= 1
        while j >= 0 and not self.exists(i, j):
            j -= 1
        return j
    
    def vertex_status(self, i):
        """获取顶点i的状态"""
        if 0 <= i < self.n:
            return self.vertices[i].status
        return None
    
    def set_vertex_status(self, i, status):
        """设置顶点i的状态"""
        if 0 <= i < self.n:
            self.vertices[i].status = status
    
    def d_time(self, i):
        """获取顶点i的发现时间"""
        if 0 <= i < self.n:
            return self.vertices[i].d_time
        return -1
    
    def set_d_time(self, i, time):
        """设置顶点i的发现时间"""
        if 0 <= i < self.n:
            self.vertices[i].d_time = time
    
    def f_time(self, i):
        """获取顶点i的完成时间"""
        if 0 <= i < self.n:
            return self.vertices[i].f_time
        return -1
    
    def set_f_time(self, i, time):
        """设置顶点i的完成时间"""
        if 0 <= i < self.n:
            self.vertices[i].f_time = time
    
    def parent(self, i):
        """获取顶点i在遍历树中的父节点"""
        if 0 <= i < self.n:
            return self.vertices[i].parent
        return -1
    
    def set_parent(self, i, parent):
        """设置顶点i在遍历树中的父节点"""
        if 0 <= i < self.n and -1 <= parent < self.n:
            self.vertices[i].parent = parent
    
    def priority(self, i):
        """获取顶点i的优先级"""
        if 0 <= i < self.n:
            return self.vertices[i].priority
        return sys.maxsize
    
    def set_priority(self, i, priority):
        """设置顶点i的优先级"""
        if 0 <= i < self.n:
            self.vertices[i].priority = priority
    
    def insert_vertex(self, data=None):
        """插入一个新顶点，返回新顶点的索引"""
        # 在顶点列表中添加新顶点
        self.vertices.append(Vertex(data))
        
        # 为新顶点在邻接矩阵中添加一行
        self.edges.append([None] * self.n)  # 新行长度为 n（将在下面扩展为 n+1）
        
        self.n += 1
        # 为已有每行添加一个新列
        for i in range(self.n):
            self.edges[i].append(None)
        
        return self.n - 1  # 返回新顶点的索引
    
    def remove_vertex(self, i):
        """删除顶点i及其相关的所有边"""
        if not (0 <= i < self.n):
            return False
        
        # 删除与顶点i相关的所有边
        for j in range(self.n):
            if self.exists(i, j):
                del self.edges[i][j]
                self.edges[i][j] = None
                self.vertices[j].in_degree -= 1
                self.e -= 1
            
            if self.exists(j, i):
                del self.edges[j][i]
                self.edges[j][i] = None
                self.vertices[j].out_degree -= 1
                self.e -= 1
        
        # 从顶点列表中删除顶点i
        del self.vertices[i]
        
        # 从邻接矩阵中删除第i行
        del self.edges[i]
        
        # 从邻接矩阵中删除第i列
        for j in range(self.n - 1):
            del self.edges[j][i]
        
        self.n -= 1
        return True
    
    def insert_edge(self, i, j, data=None, weight=0):
        """在顶点i和j之间插入一条边"""
        if not (0 <= i < self.n and 0 <= j < self.n) or self.exists(i, j):
            return False
        
        self.edges[i][j] = Edge(data, weight)
        self.vertices[i].out_degree += 1
        self.vertices[j].in_degree += 1
        self.e += 1
        return True
    
    def remove_edge(self, i, j):
        """删除顶点i和j之间的边"""
        if not self.exists(i, j):
            return False
        
        del self.edges[i][j]
        self.edges[i][j] = None
        self.vertices[i].out_degree -= 1
        self.vertices[j].in_degree -= 1
        self.e -= 1
        return True
    
    def input_matrix(self, matrix, vertex_data=None):
        """从权重矩阵输入图"""
        if not matrix or not all(len(row) == len(matrix) for row in matrix):
            return False  # 确保输入是方阵
        
        size = len(matrix)
        if vertex_data is not None and len(vertex_data) < size:
            return False  # 顶点数据长度不足
        
        # 清空现有图
        self.vertices = []
        self.edges = []
        self.n = 0
        self.e = 0
        
        # 插入顶点
        if vertex_data is None:
            vertex_data = [i for i in range(size)]
        
        for data in vertex_data[:size]:
            self.insert_vertex(data)
        
        # 插入边
        for i in range(size):
            for j in range(size):
                if matrix[i][j] != 0:  # 假设0表示没有边
                    self.insert_edge(i, j, weight=matrix[i][j])
        
        return True
    
    def output_matrix(self):
        """输出邻接矩阵（权重）"""
        matrix = []
        for i in range(self.n):
            row = []
            for j in range(self.n):
                if self.exists(i, j):
                    row.append(self.edge_weight(i, j))
                else:
                    row.append(0)
            matrix.append(row)
        return matrix
    
    def print_matrix(self):
        """打印邻接矩阵"""
        matrix = self.output_matrix()
        for row in matrix:
            print(row)
    
    def print_vertex_info(self):
        """打印所有顶点信息"""
        for i in range(self.n):
            print(f"Vertex {i}:")
            print(f"  Data: {self.vertex_data(i)}")
            print(f"  In degree: {self.in_degree(i)}")
            print(f"  Out degree: {self.out_degree(i)}")
            print(f"  Status: {self.vertex_status(i)}")
            print(f"  Discovery time: {self.d_time(i)}")
            print(f"  Finish time: {self.f_time(i)}")
            print(f"  Parent: {self.parent(i)}")
            print(f"  Priority: {self.priority(i)}")

# 示例用法
if __name__ == "__main__":
    # 创建一个图
    graph = GraphMatrix()
    
    # 插入顶点
    v0 = graph.insert_vertex("A")
    v1 = graph.insert_vertex("B")
    v2 = graph.insert_vertex("C")
    v3 = graph.insert_vertex("D")
    
    # 插入边
    graph.insert_edge(v0, v1, weight=2)
    graph.insert_edge(v0, v2, weight=3)
    graph.insert_edge(v1, v2, weight=1)
    graph.insert_edge(v1, v3, weight=4)
    graph.insert_edge(v2, v3, weight=5)
    
    print("邻接矩阵:")
    graph.print_matrix()
    
    print("\n顶点信息:")
    graph.print_vertex_info()
    
    # 测试从矩阵输入
    weight_matrix = [
        [0, 2, 3, 0],
        [0, 0, 1, 4],
        [0, 0, 0, 5],
        [0, 0, 0, 0]
    ]
    
    vertex_data = ["A", "B", "C", "D"]
    new_graph = GraphMatrix()
    new_graph.input_matrix(weight_matrix, vertex_data)
    
    print("\n从矩阵输入的邻接矩阵:")
    new_graph.print_matrix()
    
    # 测试删除边
    new_graph.remove_edge(0, 1)
    print("\n删除边(0,1)后的邻接矩阵:")
    new_graph.print_matrix()
    
    # 测试删除顶点
    new_graph.remove_vertex(1)
    print("\n删除顶点1后的邻接矩阵:")
    new_graph.print_matrix()

邻接矩阵:
[0, 2, 3, 0]
[0, 0, 1, 4]
[0, 0, 0, 5]
[0, 0, 0, 0]

顶点信息:
Vertex 0:
  Data: A
  In degree: 0
  Out degree: 2
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807
Vertex 1:
  Data: B
  In degree: 1
  Out degree: 2
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807
Vertex 2:
  Data: C
  In degree: 2
  Out degree: 1
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807
Vertex 3:
  Data: D
  In degree: 2
  Out degree: 0
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807

从矩阵输入的邻接矩阵:
[0, 2, 3, 0]
[0, 0, 1, 4]
[0, 0, 0, 5]
[0, 0, 0, 0]

删除边(0,1)后的邻接矩阵:
[0, 0, 0, 0]
[0, 0, 1, 4]
[0, 0, 0, 5]
[0, 0, 0, 0]

删除顶点1后的邻接矩阵:
[0, 0, 0]
[0, 0, 5]
[0, 0, 0]


## 图的遍历
图遍历是大部分图算法的主体框架，与树的遍历类似，图的每个顶点被访问一次且仅一次,图的所有边也被访问一次且仅访问一次（对边进行分类，教材要求）,图的遍历是将图转化为树或森林的过程（非线性到半线性结构）,遍历中组成树的一类边叫“树边”，与相应顶点构成遍历树,其它各种边提供原图的重要信息，包括环路信息等,强调对特定状态顶点的甄别与查找，故称“图搜索”,遍历包括：广度优先搜索(BFS)、深度优先搜索(DFS)，皆可在O(n+e)时间内完成
### BFS
回顾树的遍历，树的广度优先从每一层开始：
![BFStree](./sources/BFStree.png)
- 越早访问的顶点，其邻域顶点越优先被访问
- 使用队列进行缓存：反复从队列中取出队头访问，并将其未被访问的邻域顶点入队
- 必须对顶点进行标记，已访问过的顶点不再入队
![BFSgraph](./sources/BFSgraph.png)
### DFS
回顾树的遍历，树的先序，中序，后序都是一种DFS,只是访问时间不同而已：
![DFStree](./sources/DFStree.png)
- 图节点的发现与访问(递归实现）
    - 优先选取最后一个被访问到的顶点的邻居进行访问
    - 各顶点被访问到（将顶点标记为DISCOVERED）的次序，类似于树的先序遍历；各顶点被访问完（将顶点标记为VISITED）的次序，类似于树的后序遍历
    - 教材要求记录访问到的时间 dTime （DISCOVERED）及访问完的时间 fTime ( VISITED )

In [3]:
from collections import deque
def bfs(graph, s):
    """广度优先搜索，从顶点s开始"""
    if not (0 <= s < graph.n):
        return [], {}
    
    # 初始化顶点状态
    for i in range(graph.n):
        graph.set_vertex_status(i, VStatus.UNDISCOVERED)
        graph.set_d_time(i, -1)
        graph.set_parent(i, -1)
    
    # 初始化结果
    traversal_order = []
    parent_map = {}
    queue = deque()
    clock = 0
    
    # 从起点开始
    graph.set_vertex_status(s, VStatus.DISCOVERED)
    graph.set_d_time(s, clock)
    queue.append(s)
    traversal_order.append(s)
    
    while queue:
        v = queue.popleft()
        clock += 1
        
        # 遍历邻接顶点
        j = graph.first_neighbor(v)
        while j > -1:
            if graph.vertex_status(j) == VStatus.UNDISCOVERED:
                graph.set_vertex_status(j, VStatus.DISCOVERED)
                graph.set_d_time(j, clock)
                graph.set_parent(j, v)
                graph.set_edge_type(v, j, EType.TREE)
                queue.append(j)
                traversal_order.append(j)
                parent_map[j] = v
            j = graph.next_neighbor(v, j)
        
        graph.set_vertex_status(v, VStatus.VISITED)
    
    return traversal_order, parent_map

def dfs(graph, s):
    """深度优先搜索，从顶点s开始"""
    if not (0 <= s < graph.n):
        return [], {}
    
    # 初始化顶点状态
    for i in range(graph.n):
        graph.set_vertex_status(i, VStatus.UNDISCOVERED)
        graph.set_d_time(i, -1)
        graph.set_f_time(i, -1)
        graph.set_parent(i, -1)
    
    # 初始化结果
    traversal_order = []
    parent_map = {}
    clock = [0]  # 使用列表模拟可变全局时间戳
    
    def dfs_visit(v):
        nonlocal clock
        graph.set_vertex_status(v, VStatus.DISCOVERED)
        graph.set_d_time(v, clock[0])
        traversal_order.append(v)
        clock[0] += 1
        
        # 遍历邻接顶点
        j = graph.first_neighbor(v)
        while j > -1:
            if graph.vertex_status(j) == VStatus.UNDISCOVERED:
                graph.set_parent(j, v)
                graph.set_edge_type(v, j, EType.TREE)
                parent_map[j] = v
                dfs_visit(j)
            elif graph.vertex_status(j) == VStatus.DISCOVERED:
                graph.set_edge_type(v, j, EType.BACKWARD)
            elif graph.d_time(v) < graph.d_time(j):
                graph.set_edge_type(v, j, EType.FORWARD)
            else:
                graph.set_edge_type(v, j, EType.CROSS)
            j = graph.next_neighbor(v, j)
        
        graph.set_vertex_status(v, VStatus.VISITED)
        graph.set_f_time(v, clock[0])
        clock[0] += 1
    
    dfs_visit(s)
    return traversal_order, parent_map

# 示例用法
if __name__ == "__main__":
    # 创建一个图
    graph = GraphMatrix()
    
    # 插入顶点
    v0 = graph.insert_vertex("A")
    v1 = graph.insert_vertex("B")
    v2 = graph.insert_vertex("C")
    v3 = graph.insert_vertex("D")
    
    # 插入边
    graph.insert_edge(v0, v1, weight=2)
    graph.insert_edge(v0, v2, weight=3)
    graph.insert_edge(v1, v2, weight=1)
    graph.insert_edge(v1, v3, weight=4)
    graph.insert_edge(v2, v3, weight=5)
    
    print("邻接矩阵:")
    graph.print_matrix()
    
    print("\n顶点信息（初始）:")
    graph.print_vertex_info()
    
    # 测试 BFS
    print("\nBFS 从顶点0开始:")
    bfs_order, bfs_parents = bfs(graph, 0)
    print("遍历顺序:", [graph.vertex_data(i) for i in bfs_order])
    print("父节点关系:", {graph.vertex_data(k): graph.vertex_data(v) for k, v in bfs_parents.items()})
    print("\n顶点信息（BFS 后）:")
    graph.print_vertex_info()
    
    # 测试 DFS
    print("\nDFS 从顶点0开始:")
    dfs_order, dfs_parents = dfs(graph, 0)
    print("遍历顺序:", [graph.vertex_data(i) for i in dfs_order])
    print("父节点关系:", {graph.vertex_data(k): graph.vertex_data(v) for k, v in dfs_parents.items()})
    print("\n顶点信息（DFS 后）:")
    graph.print_vertex_info()
    
    # 测试从矩阵输入
    weight_matrix = [
        [0, 2, 3, 0],
        [0, 0, 1, 4],
        [0, 0, 0, 5],
        [0, 0, 0, 0]
    ]
    
    vertex_data = ["A", "B", "C", "D"]
    new_graph = GraphMatrix()
    new_graph.input_matrix(weight_matrix, vertex_data)
    
    print("\n从矩阵输入的邻接矩阵:")
    new_graph.print_matrix()
    
    # 测试 BFS 和 DFS 在新图上
    print("\nBFS 从顶点0开始（新图）:")
    bfs_order, bfs_parents = bfs(new_graph, 0)
    print("遍历顺序:", [new_graph.vertex_data(i) for i in bfs_order])
    print("父节点关系:", {new_graph.vertex_data(k): new_graph.vertex_data(v) for k, v in bfs_parents.items()})
    
    print("\nDFS 从顶点0开始（新图）:")
    dfs_order, dfs_parents = dfs(new_graph, 0)
    print("遍历顺序:", [new_graph.vertex_data(i) for i in dfs_order])
    print("父节点关系:", {new_graph.vertex_data(k): new_graph.vertex_data(v) for k, v in bfs_parents.items()})

Exception ignored in: <function GraphMatrix.__del__ at 0x0000020C59619480>
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_7992\2644747120.py", line 49, in __del__
IndexError: list index out of range
Exception ignored in: <function GraphMatrix.__del__ at 0x0000020C59619480>
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_7992\2644747120.py", line 49, in __del__
IndexError: list index out of range


邻接矩阵:
[0, 2, 3, 0]
[0, 0, 1, 4]
[0, 0, 0, 5]
[0, 0, 0, 0]

顶点信息（初始）:
Vertex 0:
  Data: A
  In degree: 0
  Out degree: 2
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807
Vertex 1:
  Data: B
  In degree: 1
  Out degree: 2
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807
Vertex 2:
  Data: C
  In degree: 2
  Out degree: 1
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807
Vertex 3:
  Data: D
  In degree: 2
  Out degree: 0
  Status: VStatus.UNDISCOVERED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807

BFS 从顶点0开始:
遍历顺序: ['A', 'C', 'B', 'D']
父节点关系: {'C': 'A', 'B': 'A', 'D': 'C'}

顶点信息（BFS 后）:
Vertex 0:
  Data: A
  In degree: 0
  Out degree: 2
  Status: VStatus.VISITED
  Discovery time: 0
  Finish time: -1
  Parent: -1
  Priority: 9223372036854775807
Vertex 1:
  Dat

## 最小支撑树（最小生成树）
- 连通图G的某一无环连通子图T若覆盖G中所有的顶点，则称作G的一棵支撑树或生成树
- n个顶点的连通网络的生成树有n个顶点、n-1条边
- 不能使用产生回路的边
- 各边上的权值的总和达到最小
- 应用：假设有一个网络，用以表示 n 个城市之间架设通信线路，边上的权值代表架设通信线路的成本。如何架设才能使线路架设的成本达到最小？

### 生成算法：
1. 普里姆算法
    - 顶点集V和顶点集U构成G={V,U}的一个割（cut）
    - 最小生成树总是采用联接每一割的最短跨越边（贪婪迭代法）
    - 从一个点开始扩充，每次找最短边，知道找到所有点；

In [4]:
from heapq import heappush, heappop
import sys

def prim(graph, s):
    """Prim 算法实现最小生成树，从顶点 s 开始"""
    if not (0 <= s < graph.n):
        return [], 0
    
    # 初始化顶点状态
    for i in range(graph.n):
        graph.set_vertex_status(i, VStatus.UNDISCOVERED)
        graph.set_parent(i, -1)
        graph.set_priority(i, sys.maxsize)
    
    # 起点优先级为 0
    graph.set_priority(s, 0)
    pq = []  # 优先队列 (priority, vertex)
    heappush(pq, (0, s))
    
    mst_edges = []  # MST 边列表 (parent, child, weight)
    total_weight = 0
    visited = set()  # 已访问顶点
    
    while pq:
        pri, u = heappop(pq)
        if u in visited:
            continue
        visited.add(u)
        graph.set_vertex_status(u, VStatus.VISITED)
        
        # 添加到 MST 的边
        if graph.parent(u) != -1:
            mst_edges.append((graph.parent(u), u, pri))
            total_weight += pri
        
        # 更新邻接顶点
        j = graph.first_neighbor(u)
        while j > -1:
            if graph.vertex_status(j) != VStatus.VISITED:
                weight = graph.edge_weight(u, j)
                if weight is not None and weight < graph.priority(j):
                    graph.set_priority(j, weight)
                    graph.set_parent(j, u)
                    graph.set_edge_type(u, j, EType.TREE)
                    heappush(pq, (weight, j))
            j = graph.next_neighbor(u, j)
    
    return mst_edges, total_weight

# 示例图：顶点 A(0), B(1), C(2), D(3)
# 边（无向）：A-B:2, A-C:3, B-C:1, B-D:4, C-D:5
graph = GraphMatrix()

# 插入顶点
v0 = graph.insert_vertex("A")
v1 = graph.insert_vertex("B")
v2 = graph.insert_vertex("C")
v3 = graph.insert_vertex("D")

# 插入边（模拟无向图，添加双向边）
graph.insert_edge(v0, v1, weight=2)
graph.insert_edge(v1, v0, weight=2)
graph.insert_edge(v0, v2, weight=3)
graph.insert_edge(v2, v0, weight=3)
graph.insert_edge(v1, v2, weight=1)
graph.insert_edge(v2, v1, weight=1)
graph.insert_edge(v1, v3, weight=4)
graph.insert_edge(v3, v1, weight=4)
graph.insert_edge(v2, v3, weight=5)
graph.insert_edge(v3, v2, weight=5)

# 调用 Prim 算法，从顶点 0 开始
mst_edges, total_weight = prim(graph, 0)

# 输出结果
print("最小生成树边列表:")
for parent, child, weight in mst_edges:
    print(f"{graph.vertex_data(parent)} -> {graph.vertex_data(child)} (权重: {weight})")

print(f"总权重: {total_weight}")

# 打印顶点信息（验证 parent 和 priority）
print("\n顶点信息（Prim 后）:")
graph.print_vertex_info()

Exception ignored in: <function GraphMatrix.__del__ at 0x0000020C59619480>
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_7992\2644747120.py", line 49, in __del__
IndexError: list index out of range


最小生成树边列表:
A -> B (权重: 2)
B -> C (权重: 1)
B -> D (权重: 4)
总权重: 7

顶点信息（Prim 后）:
Vertex 0:
  Data: A
  In degree: 2
  Out degree: 2
  Status: VStatus.VISITED
  Discovery time: -1
  Finish time: -1
  Parent: -1
  Priority: 0
Vertex 1:
  Data: B
  In degree: 3
  Out degree: 3
  Status: VStatus.VISITED
  Discovery time: -1
  Finish time: -1
  Parent: 0
  Priority: 2
Vertex 2:
  Data: C
  In degree: 3
  Out degree: 3
  Status: VStatus.VISITED
  Discovery time: -1
  Finish time: -1
  Parent: 1
  Priority: 1
Vertex 3:
  Data: D
  In degree: 2
  Out degree: 2
  Status: VStatus.VISITED
  Discovery time: -1
  Finish time: -1
  Parent: 1
  Priority: 4


## 最短路径树
### 最短路径
- 如果从图中某一顶点（称为源点）到另一顶点（称为终点）的路径可能不止一条，如何找到一条路径使得沿此路径上各边上的权值总和达到最小
### 什么是最短路径树：
- 单调性：最短路径的任意前缀也是最短路径；S到v的最短路径经过u，则沿着该路径从S到u也是u的最短路径（可反证）
- 无环性：S到图中其它各点的最短路径的集合必无环
- 最短路径树（SPT, Shortest Path Tree）
### 算法：
1. Dijkstra算法
    - 边上权值**非负情形**的**单源**最短路径问题
        - 给定一个带权有向图G与源点s，求从s到G中其他顶点的最短路径
        - 限定各边上的权值大于或等于0
        - 按路径长度的递增次序,   逐步产生最短路径
        - 首先求出长度最短的一条最短路径(s,u)，更新SPT的顶点集及s到其他各边的最短距离（更新u的邻域），再求出s到其它顶点长度次短的一条最短路径，依次类推，直到所有顶点进入SPT集合

2. Bellman和Ford算法
    - 边上权值为**任意值**的**单源**最短路径问题
    - 限制条件：图中不能包含负权回路（回路的权值和为负)
        - 没有负权和回路时，n个顶点的图中任意两个顶点之间如果存在最短路径，此路径最多有n-1条边
        - 构造最短路径长度数组序列$dist_1[v], dist_2[v], …, dist_{n-1}[v]$。
        - $dist_1 [v]$是从源点u到终点v的只经过一条边的最短路径的长度$dist_1 [v] = Edge[u][v]$
        - $dist_k [v]$是从源点u出发最多经过k条边到达终点v的最短路径长度
        - 算法的最终目的是计算出 $dist_{n-1}[v]$，采用递推计算
        - 算法需判断是否存在负权和回路，可在n-1次迭代后再做一迭代，若某节点最小距离仍能更新，则存在负值和回路
3. Floyd算法
    - **所有顶点间**的最短路径

In [ ]:
import sys
from heapq import heappush, heappop

def dijkstra(graph, s):
    """Dijkstra 算法：单源最短路径（非负权重）"""
    if not (0 <= s < graph.n):
        return {}, {}  # 返回距离字典和父节点字典
    
    # 初始化
    dist = {i: float('inf') for i in range(graph.n)}
    parent = {i: -1 for i in range(graph.n)}
    dist[s] = 0
    pq = []  # 优先队列 (dist, vertex)
    heappush(pq, (0, s))
    
    while pq:
        d, u = heappop(pq)
        if d > dist[u]:
            continue
        
        # 遍历邻接顶点
        j = graph.first_neighbor(u)
        while j > -1:
            weight = graph.edge_weight(u, j)
            if weight is not None and weight >= 0:  # 确保非负
                new_dist = dist[u] + weight
                if new_dist < dist[j]:
                    dist[j] = new_dist
                    parent[j] = u
                    heappush(pq, (new_dist, j))
            j = graph.next_neighbor(u, j)
    
    # 更新图属性（可选，使用 priority 和 parent）
    for i in range(graph.n):
        graph.set_priority(i, dist[i] if dist[i] != float('inf') else sys.maxsize)
        graph.set_parent(i, parent[i])
    
    return dist, parent

def bellman_ford(graph, s):
    """Bellman-Ford 算法：单源最短路径（允许负权重，无负环）"""
    if not (0 <= s < graph.n):
        return {}, {}  # 返回距离字典和父节点字典
    
    # 初始化
    dist = {i: float('inf') for i in range(graph.n)}
    parent = {i: -1 for i in range(graph.n)}
    dist[s] = 0
    
    # 放松 |V|-1 次
    for _ in range(graph.n - 1):
        for u in range(graph.n):
            j = graph.first_neighbor(u)
            while j > -1:
                weight = graph.edge_weight(u, j)
                if weight is not None and dist[u] != float('inf'):
                    new_dist = dist[u] + weight
                    if new_dist < dist[j]:
                        dist[j] = new_dist
                        parent[j] = u
                j = graph.next_neighbor(u, j)
    
    # 检查负环（可选，这里假设无负环）
    
    # 更新图属性（可选）
    for i in range(graph.n):
        graph.set_priority(i, dist[i] if dist[i] != float('inf') else sys.maxsize)
        graph.set_parent(i, parent[i])
    
    return dist, parent

def floyd_warshall(graph):
    """Floyd-Warshall 算法：所有顶点间最短路径，带 P 矩阵"""
    # 初始化距离矩阵和前驱矩阵
    dist = [[float('inf')] * graph.n for _ in range(graph.n)]
    P = [[None] * graph.n for _ in range(graph.n)]
    
    for i in range(graph.n):
        dist[i][i] = 0
        j = graph.first_neighbor(i)
        while j > -1:
            weight = graph.edge_weight(i, j)
            if weight is not None:
                dist[i][j] = weight
                P[i][j] = i  # i 是 j 的前驱
            j = graph.next_neighbor(i, j)
    
    # 动态规划
    for k in range(graph.n):
        for i in range(graph.n):
            for j in range(graph.n):
                if dist[i][k] != float('inf') and dist[k][j] != float('inf'):
                    new_dist = dist[i][k] + dist[k][j]
                    if new_dist < dist[i][j]:
                        dist[i][j] = new_dist
                        P[i][j] = P[k][j]  # 更新前驱
    
    return dist, P

def reconstruct_path(P, i, j):
    """根据 P 矩阵重构从 i 到 j 的最短路径"""
    if P[i][j] is None and i != j:
        return []  # 无路径
    path = []
    curr = j
    while curr is not None and curr != i:
        path.append(curr)
        curr = P[i][curr]
    if curr == i:
        path.append(i)
    return path[::-1]  # 逆序返回路径

# 示例用法
if __name__ == "__main__":
    graph = GraphMatrix()
    
    # 插入顶点
    v0 = graph.insert_vertex("A")
    v1 = graph.insert_vertex("B")
    v2 = graph.insert_vertex("C")
    v3 = graph.insert_vertex("D")
    
    # 插入边（有向，权重非负）
    graph.insert_edge(v0, v1, weight=2)
    graph.insert_edge(v0, v2, weight=3)
    graph.insert_edge(v1, v2, weight=1)
    graph.insert_edge(v1, v3, weight=4)
    graph.insert_edge(v2, v3, weight=5)
    
    # Dijkstra 示例（从 A 开始）
    dist_dijk, parent_dijk = dijkstra(graph, 0)
    print("Dijkstra 最短距离:", {graph.vertex_data(k): v for k, v in dist_dijk.items()})
    print("Dijkstra 父节点:", {graph.vertex_data(k): graph.vertex_data(v) if v != -1 else None for k, v in parent_dijk.items()})
    
    # Bellman-Ford 示例（从 A 开始）
    dist_bf, parent_bf = bellman_ford(graph, 0)
    print("\nBellman-Ford 最短距离:", {graph.vertex_data(k): v for k, v in dist_bf.items()})
    print("Bellman-Ford 父节点:", {graph.vertex_data(k): graph.vertex_data(v) if v != -1 else None for k, v in parent_bf.items()})
    
    # Floyd-Warshall 示例
    dist_floyd, P_floyd = floyd_warshall(graph)
    vertex_labels = [graph.vertex_data(i) for i in range(graph.n)]
    
    print("\nFloyd-Warshall 距离矩阵:")
    for i in range(graph.n):
        row = {vertex_labels[j]: dist_floyd[i][j] if dist_floyd[i][j] != float('inf') else 'inf' for j in range(graph.n)}
        print(f"从 {vertex_labels[i]}: {row}")
    
    print("\nFloyd-Warshall 前驱矩阵 (P):")
    for i in range(graph.n):
        row = {vertex_labels[j]: vertex_labels[P_floyd[i][j]] if P_floyd[i][j] is not None else 'None' for j in range(graph.n)}
        print(f"从 {vertex_labels[i]}: {row}")
    
    # 测试路径重构
    print("\n最短路径示例:")
    for i in range(graph.n):
        for j in range(graph.n):
            if i != j and dist_floyd[i][j] != float('inf'):
                path = reconstruct_path(P_floyd, i, j)
                path_labels = [graph.vertex_data(v) for v in path]
                print(f"从 {vertex_labels[i]} 到 {vertex_labels[j]} 的最短路径: {' -> '.join(path_labels)} (距离: {dist_floyd[i][j]})")

Exception ignored in: <function GraphMatrix.__del__ at 0x0000020C59619480>
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Temp\ipykernel_7992\2644747120.py", line 49, in __del__
IndexError: list index out of range


Dijkstra 最短距离: {'A': 0, 'B': 2, 'C': 3, 'D': 6}
Dijkstra 父节点: {'A': None, 'B': 'A', 'C': 'A', 'D': 'B'}

Bellman-Ford 最短距离: {'A': 0, 'B': 2, 'C': 3, 'D': 6}
Bellman-Ford 父节点: {'A': None, 'B': 'A', 'C': 'A', 'D': 'B'}

Floyd-Warshall 距离矩阵:
从 A: {'A': 0, 'B': 2, 'C': 3, 'D': 6}
从 B: {'A': 'inf', 'B': 0, 'C': 1, 'D': 4}
从 C: {'A': 'inf', 'B': 'inf', 'C': 0, 'D': 5}
从 D: {'A': 'inf', 'B': 'inf', 'C': 'inf', 'D': 0}

Floyd-Warshall 前驱矩阵 (P):
从 A: {'A': 'None', 'B': 'A', 'C': 'A', 'D': 'B'}
从 B: {'A': 'None', 'B': 'None', 'C': 'B', 'D': 'B'}
从 C: {'A': 'None', 'B': 'None', 'C': 'None', 'D': 'C'}
从 D: {'A': 'None', 'B': 'None', 'C': 'None', 'D': 'None'}

最短路径示例:
从 A 到 B 的最短路径: A -> B (距离: 2)
从 A 到 C 的最短路径: A -> C (距离: 3)
从 A 到 D 的最短路径: A -> B -> D (距离: 6)
从 B 到 C 的最短路径: B -> C (距离: 1)
从 B 到 D 的最短路径: B -> D (距离: 4)
从 C 到 D 的最短路径: C -> D (距离: 5)


## 图的优先级搜索
- BFS优先考虑当前所有被发现点中，最早被发现的点；
- DFS优先考虑当前所有被发现点中，最后被发现的点；
- Prim和Dijkstra考虑当前被发现点中，优先级最高的点；
- Prim的优先级更新：=父亲与该点之间的路径权重；
- Dijkstra的优先级更新：+父亲与该点之间的路径权重；
- 每种策略等效于选取当前约定的优先级策略下的最高者
- 优先级搜索提供统一的框架


## AOV网与拓扑排序总结
(不怎么重要，AI生成)

### 1. AOV网的概念
- **AOV网 (Activity On Vertex Network)**  
  - 一种有向图模型。  
  - **顶点 (Vertex)** 表示活动 (Activity)。  
  - **有向边 (Arc)** 表示活动之间的优先关系（依赖约束）。  
  - 无向边不存在。  

- **特点**  
  1. 必须是 **有向无环图 (DAG)**，即不存在回路。  
  2. 边 `<vi, vj>` 表示活动 `vi` 必须先于 `vj` 执行。  
  3. 用于描述任务调度、课程安排、工程步骤等依赖关系。  

---

### 2. AOV网与环路问题
- **若AOV网中存在环路** → 无法进行拓扑排序。  
- 环路意味着某些活动之间存在循环依赖，调度无法完成。  

---

### 3. 拓扑排序 (Topological Sorting)
- **定义**  
  对一个 **有向无环图 (DAG)** 的所有顶点排成一个线性序列，使得：  
  - 若存在边 `<vi, vj>`，则在序列中 `vi` 出现在 `vj` 之前。  

- **结果**  
  - 拓扑排序序列不唯一。  
  - 存在拓扑序列的充要条件：图为 **DAG**。  

---

### 4. 拓扑排序的常用算法

#### 方法一：入度法 (Kahn算法)
1. 统计所有顶点的 **入度**。  
2. 选取入度为 `0` 的顶点输出，并删除其相关边。  
3. 更新剩余顶点的入度，重复步骤 2。  
4. 若最终输出顶点数 < 总顶点数 → 图中存在环。  

#### 方法二：DFS法 (逆拓扑序)
1. 对图进行深度优先遍历 (DFS)。  
2. 在递归返回时，将顶点压入栈。  
3. 最终栈中顶点依次弹出 → 拓扑序列。  
4. 若DFS中检测到回边 → 图中有环。  

---

### 5. 应用场景
- 课程先修关系安排。  
- 程序编译依赖分析。  
- 任务调度与工作流管理。  
- 工程项目中的步骤安排 (PERT图)。  